In [1]:
import os
import sys
import re
import pandas as pd
import commands
import ast
from StringIO import StringIO
import itertools
import pyspark.sql.functions
from pyspark.sql.functions import col
from datetime import datetime
from collections import Counter
import re
import numpy as np
#from pyspark.sql import SparkSession
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn-client \
                                         --executor-memory 4g --executor-cores 5 --driver-memory 16g"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.0-292
      /_/

Using Python version 2.7.15 (default, May  1 2018 23:32:55)
SparkSession available as 'spark'.


In [2]:
database="dsc60230_gcct_tz_db"
sqlContext.sql("use "+database)

DataFrame[]

In [3]:
#Writes schema to excel file. All in one sheet uses the list of tables
df= pd.DataFrame(columns = ['col_name','data_type','table'])
writer = pd.ExcelWriter("GCCT_schema_Jan19.xlsx", engine = 'xlsxwriter')
tab_list=['account','activityparty','annotation','appointment','businessunit','calendar','calendarrule','contact','cxlvhlp_chatactivity','cxlvhlp_chatqueuestatistic','cxlvhlp_surveyitem','email','fax','gcct_accountresponsibleagent','gcct_additionalsymptomcodes','gcct_addportalmessage','gcct_arbitrationclaimprocessing','gcct_buybackevaluationmilestones','gcct_caseassignment','gcct_caseclassification','gcct_casedispositiontype','gcct_coachback','gcct_country','gcct_customersatisfactiontools','gcct_delegationofauthority','gcct_demandltrtpsmclaimsprocessing','gcct_doaprogramcode','gcct_documentcustomerrecontact','gcct_engine','gcct_executive','gcct_executiveliaison','gcct_fieldinvolvementassistancerequest','gcct_fmcccontract','gcct_fulfillmentvendor','gcct_genericinformation','gcct_genericinformationtopic','gcct_generictopic','gcct_goodfaithreview','gcct_knownpartsdelay','gcct_lemonlawcriteria','gcct_loanercost','gcct_loyaltyassistance','gcct_loyaltycostdetails','gcct_loyaltyprogramcode','gcct_material','gcct_materialrequest','gcct_materialrequestdetail','gcct_offer','gcct_offerdetail','gcct_onlinegaragevehicles','gcct_partsorderstatus','gcct_partsordersystems','gcct_partssmeassistancerequest','gcct_pcarequest','gcct_priordealerdecision','gcct_qaincident','gcct_qamonitoring','gcct_queuepriority','gcct_ravprocessingmilestones','gcct_reactdata','gcct_reacttransmission','gcct_reasoncode','gcct_recall','gcct_region','gcct_rentaldetail','gcct_slaconfiguration','gcct_sms','gcct_socialmediaconversation','gcct_socialmediamessage','gcct_socialmediaprofile','gcct_specialloanercode','gcct_state','gcct_surveyconfiguration','gcct_symptomclassification','gcct_tasktype','gcct_technicalassistancerequest','gcct_testdriverequest','gcct_timezone','gcct_userdelegationofauthority','gcct_vehicle','gcct_vehiclebrand','gcct_vehicleclass','gcct_vehicleloyaltyallowance','gcct_vehicleoffroad','gcct_vehicleowner','gcct_warrantyhistory','gcct_warrantyloanerpartsdelay','gcct_warrantyloanerrequest','gcct_warrantyloanerrequesthistory','globaloptionsetmetadata','incident','incidentresolution','letter','msdyn_answer','msdyn_question','msdyn_questionresponse','msdyn_survey','msdyn_surveyinvite','msdyn_surveyresponse','optionsetmetadata','phonecall','queue','queueitem','sla','slaitem','slakpiinstance','socialactivity','socialprofile','statemetadata','statusmetadata','systemuser','task','team','teammembership','territory']
#for m in sqlContext.tables(database).select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%dup%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").collect():
for k in tab_list:
            #k=m.asDict().values()[0].encode('utf-8')
            g1=sqlCtx.sql('desc '+k).toPandas()
            g1['table']=k
            g1=g1[['col_name','data_type','table']]
            df=df.append(g1, ignore_index=True)
            


df.to_excel(writer, sheet_name="sheet1", index=False)
writer.save()
writer.close()

In [273]:
#tab_list=['account','activityparty','annotation','appointment','businessunit','calendar','calendarrule','contact','cxlvhlp_chatactivity','cxlvhlp_chatqueuestatistic','cxlvhlp_surveyitem','email','fax','gcct_accountresponsibleagent','gcct_additionalsymptomcodes','gcct_addportalmessage','gcct_arbitrationclaimprocessing','gcct_buybackevaluationmilestones','gcct_caseassignment','gcct_caseclassification','gcct_casedispositiontype','gcct_coachback','gcct_country','gcct_customersatisfactiontools','gcct_delegationofauthority','gcct_demandltrtpsmclaimsprocessing','gcct_doaprogramcode','gcct_documentcustomerrecontact','gcct_engine','gcct_executive','gcct_executiveliaison','gcct_fieldinvolvementassistancerequest','gcct_fmcccontract','gcct_fulfillmentvendor','gcct_genericinformation','gcct_genericinformationtopic','gcct_generictopic','gcct_goodfaithreview','gcct_knownpartsdelay','gcct_lemonlawcriteria','gcct_loanercost','gcct_loyaltyassistance','gcct_loyaltycostdetails','gcct_loyaltyprogramcode','gcct_material','gcct_materialrequest','gcct_materialrequestdetail','gcct_offer','gcct_offerdetail','gcct_onlinegaragevehicles','gcct_partsorderstatus','gcct_partsordersystems','gcct_partssmeassistancerequest','gcct_pcarequest','gcct_priordealerdecision','gcct_qaincident','gcct_qamonitoring','gcct_queuepriority','gcct_ravprocessingmilestones','gcct_reactdata','gcct_reacttransmission','gcct_reasoncode','gcct_recall','gcct_region','gcct_rentaldetail','gcct_slaconfiguration','gcct_sms','gcct_socialmediaconversation','gcct_socialmediamessage','gcct_socialmediaprofile','gcct_specialloanercode','gcct_state','gcct_surveyconfiguration','gcct_symptomclassification','gcct_tasktype','gcct_technicalassistancerequest','gcct_testdriverequest','gcct_timezone','gcct_userdelegationofauthority','gcct_vehicle','gcct_vehiclebrand','gcct_vehicleclass','gcct_vehicleloyaltyallowance','gcct_vehicleoffroad','gcct_vehicleowner','gcct_warrantyhistory','gcct_warrantyloanerpartsdelay','gcct_warrantyloanerrequest','gcct_warrantyloanerrequesthistory','globaloptionsetmetadata','incident','incidentresolution','letter','msdyn_answer','msdyn_question','msdyn_questionresponse','msdyn_survey','msdyn_surveyinvite','msdyn_surveyresponse','optionsetmetadata','phonecall','queue','queueitem','sla','slaitem','slakpiinstance','socialactivity','socialprofile','statemetadata','statusmetadata','systemuser','task','team','teammembership','territory']
#df=pd.ExcelFile('/s/latluri1/DES/GCCT_schema_Jan19.xlsx').parse("sheet1")

In [34]:
commands.getstatusoutput("ls /s/latluri1/DES")[1].splitlines()

['DESDEV_processed.csv',
 'DESDEV.sql',
 'DESDSC_Jan25_processed.csv',
 'DESDSC_Jan25.sql',
 'DESPRE-Prod_processed.csv',
 'DESPRE-Prod.sql',
 'DES_prod_processed.csv',
 'DES_prod.sql',
 'DESQA_processed.csv',
 'DESQA.sql',
 'DESscript_20190313_processed.csv',
 'DESscript_20190313.sql',
 'dev.xlsx',
 'gcct_DES_added_cols.xlsx',
 '~$GCCT_schema_Jan19.xlsx',
 'GCCT_schema_Jan19.xlsx',
 'pre_prod.xlsx',
 '~$prod.xlsx',
 'prod.xlsx',
 'qa_mar_11.xlsx',
 'sol_test.xlsx']

# DES_prod_processed.csv
# DESQA_processed.csv
# DESDSC_Jan25_processed.csv
# DESDEV_processed.csv
# DESscript_20190313_processed.csv
# DESPRE-Prod_processed.csv',

In [13]:

old=df.apply(lambda x: x.astype(str).str.lower())
import xlrd
output_name='DESscript_20190313'
df_new=pd.read_csv('/s/latluri1/DES/DESscript_20190313_processed.csv')
new=df_new.apply(lambda x: x.astype(str).str.lower())

In [14]:
output_name

'DESscript_20190313'

In [15]:
#bit->boolean
#uniqueidentifier->string
#datetime->timestamp
#bit->boolean
#nvarchar->string
#varchar->string
#timestamp->string

In [16]:
new.columns

Index([u'TABLE_NAME', u'COLUMN_NAME', u'DATA_TYPE', u'NUMERIC_PRECISION',
       u'NUMERIC_SCALE'],
      dtype='object')

In [17]:
set(list(new['DATA_TYPE']))

{'bigint',
 'bit',
 'datetime',
 'decimal',
 'float',
 'int',
 'nvarchar',
 'timestamp',
 'uniqueidentifier'}

In [279]:
#new=df_new.loc[new['DATA_TYPE'] == 'decimal' ]
#new.loc[pd.isnull(new['NUMERIC_SCALE']),['DATA_TYPE','NUMERIC_PRECISION','NUMERIC_SCALE']]
#new.loc[pd.isnull(new['NUMERIC_PRECISION']),['DATA_TYPE','NUMERIC_PRECISION','NUMERIC_SCALE']]

In [18]:
#df_new=df_new.loc[df_new['DATA_TYPE'] == 'decimal']
def get_decimal(a):
    if(a['DATA_TYPE']=='decimal'):
        if np.logical_not(pd.isnull(a['NUMERIC_PRECISION'])):
            if np.logical_not(pd.isnull(a['NUMERIC_SCALE'])):
                return(str(a['DATA_TYPE'])+"("+str(int(float(a['NUMERIC_PRECISION'])))+","+str(int(float(a['NUMERIC_SCALE'])))+")")
            else:
                return(str(a['DATA_TYPE'])+"("+str(int(float(a['NUMERIC_PRECISION'])))+")")
        else:
            return(a['DATA_TYPE'])
    else:
        return(a['DATA_TYPE'])
    
new.loc[:,'DATA_TYPE']=new[['DATA_TYPE','NUMERIC_PRECISION','NUMERIC_SCALE']].apply(lambda r: get_decimal(r), axis=1)

In [19]:
new

,TABLE_NAME,COLUMN_NAME,DATA_TYPE,NUMERIC_PRECISION,NUMERIC_SCALE
0,account,account_identity,int,identity,1
1,account,account_rowversion,timestamp,nan,nan
2,account,id,uniqueidentifier,not,nan
3,account,sinkcreatedon,datetime,nan,nan
4,account,sinkmodifiedon,datetime,nan,nan
5,account,statecode,int,nan,nan
6,account,statuscode,int,nan,nan
7,account,customertypecode,int,nan,nan
8,account,address1_freighttermscode,int,nan,nan
9,account,address1_addresstypecode,int,nan,nan


In [20]:
set(list(new['DATA_TYPE']))

{'bigint',
 'bit',
 'datetime',
 'decimal(38,0)',
 'decimal(38,10)',
 'decimal(38,2)',
 'decimal(38,3)',
 'decimal(38,4)',
 'decimal(38,5)',
 'float',
 'int',
 'nvarchar',
 'timestamp',
 'uniqueidentifier'}

In [21]:
set(list(old['data_type']))

{'bigint',
 'boolean',
 'decimal(38,0)',
 'decimal(38,10)',
 'decimal(38,2)',
 'decimal(38,3)',
 'decimal(38,4)',
 'decimal(38,5)',
 'float',
 'int',
 'string',
 'timestamp'}

In [283]:
new.loc[new['DATA_TYPE'] == 'nvarchar']

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,CHARACTER_OCTET_LENGTH,...,DATETIME_PRECISION,CHARACTER_SET_CATALOG,CHARACTER_SET_SCHEMA,CHARACTER_SET_NAME,COLLATION_CATALOG,COLLATION_SCHEMA,COLLATION_NAME,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME
6,fordgcctintegrationdesst,dbo,gcct_region,owninguser_entitytype,7,nan,yes,nvarchar,128.0,256.0,...,nan,nan,nan,unicode,nan,nan,sql_latin1_general_cp1_ci_as,nan,nan,nan
8,fordgcctintegrationdesst,dbo,gcct_region,createdonbehalfby_entitytype,9,nan,yes,nvarchar,128.0,256.0,...,nan,nan,nan,unicode,nan,nan,sql_latin1_general_cp1_ci_as,nan,nan,nan
10,fordgcctintegrationdesst,dbo,gcct_region,owningbusinessunit_entitytype,11,nan,yes,nvarchar,128.0,256.0,...,nan,nan,nan,unicode,nan,nan,sql_latin1_general_cp1_ci_as,nan,nan,nan
12,fordgcctintegrationdesst,dbo,gcct_region,owningteam_entitytype,13,nan,yes,nvarchar,128.0,256.0,...,nan,nan,nan,unicode,nan,nan,sql_latin1_general_cp1_ci_as,nan,nan,nan
14,fordgcctintegrationdesst,dbo,gcct_region,modifiedby_entitytype,15,nan,yes,nvarchar,128.0,256.0,...,nan,nan,nan,unicode,nan,nan,sql_latin1_general_cp1_ci_as,nan,nan,nan
16,fordgcctintegrationdesst,dbo,gcct_region,createdby_entitytype,17,nan,yes,nvarchar,128.0,256.0,...,nan,nan,nan,unicode,nan,nan,sql_latin1_general_cp1_ci_as,nan,nan,nan
18,fordgcctintegrationdesst,dbo,gcct_region,modifiedonbehalfby_entitytype,19,nan,yes,nvarchar,128.0,256.0,...,nan,nan,nan,unicode,nan,nan,sql_latin1_general_cp1_ci_as,nan,nan,nan
20,fordgcctintegrationdesst,dbo,gcct_region,ownerid_entitytype,21,nan,yes,nvarchar,128.0,256.0,...,nan,nan,nan,unicode,nan,nan,sql_latin1_general_cp1_ci_as,nan,nan,nan
21,fordgcctintegrationdesst,dbo,gcct_region,createdonbehalfbyyominame,22,nan,yes,nvarchar,100.0,200.0,...,nan,nan,nan,unicode,nan,nan,sql_latin1_general_cp1_ci_as,nan,nan,nan
22,fordgcctintegrationdesst,dbo,gcct_region,owneridname,23,nan,yes,nvarchar,100.0,200.0,...,nan,nan,nan,unicode,nan,nan,sql_latin1_general_cp1_ci_as,nan,nan,nan


In [22]:
# df['BrandName'].replace(['ABC', 'AB'], 'A')
#              conversionMap.put("int"                 , "int"      );
#              conversionMap.put("uniqueidentifier"   , "string"   );
#              conversionMap.put("datetime"            , "timestamp" );
#              conversionMap.put("bit"                 , "boolean"  );
#              conversionMap.put("nvarchar"            , "string"   );
#              conversionMap.put("decimal"             , "decimal"  );
#              conversionMap.put("bigint"              , "bigint"   );
#              conversionMap.put("timestamp"           , "string"   );
#              conversionMap.put("float"               , "float"    );


new.loc[:,'DATA_TYPE']=new['DATA_TYPE'].replace('bit', 'boolean')
new.loc[:,'DATA_TYPE']=new['DATA_TYPE'].replace(['uniqueidentifier','varchar','nvarchar'], 'string')
new.loc[:,'DATA_TYPE']=new['DATA_TYPE'].replace('timestamp', 'string')
new.loc[:,'DATA_TYPE']=new['DATA_TYPE'].replace('datetime', 'timestamp')


In [23]:
set(list(new['DATA_TYPE']))

{'bigint',
 'boolean',
 'decimal(38,0)',
 'decimal(38,10)',
 'decimal(38,2)',
 'decimal(38,3)',
 'decimal(38,4)',
 'decimal(38,5)',
 'float',
 'int',
 'string',
 'timestamp'}

In [24]:
#new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])
final_1=old.merge(new,how='outer',left_on=['table','col_name'],right_on=['TABLE_NAME','COLUMN_NAME'],suffixes=('_old','new') )
final_2=final_1[['TABLE_NAME','COLUMN_NAME','table','col_name','data_type','DATA_TYPE']]
final=final_2.loc[(final_2['TABLE_NAME'].isin(tab_list)) | (final_2['table'].isin(tab_list))]
#final.loc[:,'data_type']=final.data_type.apply(lambda x: x.strip())
#final.loc[:,'DATA_TYPE']=final.DATA_TYPE.apply(lambda x: x.strip())

In [25]:
print(len(old),len(new),len(final_1))

(12341, 11441, 12539)


In [26]:
#df[['Len_old', 'Len_new']].assign(NE=df.Len_old != df.Len_new)
#df.loc[df['A'] == 'foo']
delta=final[['TABLE_NAME','COLUMN_NAME','data_type', 'DATA_TYPE']].assign(NE=final.data_type != final.DATA_TYPE)
delta = delta.dropna(subset=['data_type','DATA_TYPE'])
delta.rename(columns={'TABLE_NAME':'Table_name','COLUMN_NAME':'Column_name','data_type':'Old_data_type(DSC)','DATA_TYPE':'New_data_type(DES)'}, inplace=True)

In [27]:




writer = pd.ExcelWriter("GCCT_schema_changes_"+output_name+".xlsx", engine = 'xlsxwriter')
dtype_changes=delta.loc[delta['NE'] == True]
#removed
ignore_list=['entityid','entitystring','eventdatetime','eventtype','lastupdatedatetime','lastupdtby','message__id','process__id','shakey']
#set(list(final.loc[final['TABLE_NAME'].isnull(),]['col_name']))
#removed=final.loc[(final['TABLE_NAME'].isnull()) &( np.logical_not(final['col_name'].isin(ignore_list))),]
#added
added=final.loc[final['table'].isnull(),['TABLE_NAME','COLUMN_NAME','DATA_TYPE']]


#Tables removed
#tab_removed_1=((set(list(final_1.table)).difference(set(list(final_1.TABLE_NAME)))).difference(set(tab_list)))
#tab_removed='tables added'+str(tab_removed_1).replace("set(","").replace(")","")
#Tables added
#tab_added_1=((set(list(final_1.TABLE_NAME)).difference(set(list(final_1.table)))).difference(set(tab_list)))
#tab_added='tables added'+str(tab_added_1).replace("set(","").replace(")","")

#Tables added
tab_added_1=set(list(final_1.TABLE_NAME)).difference(set(list(final_1.table)))
tables_added= pd.DataFrame({'Tables added': list(tab_added_1)})



#Tables removed
tab_removed_1=set(list(final_1.table)).difference(set(list(final_1.TABLE_NAME)))
tab_removed=pd.DataFrame({'Tables Removed':list(tab_removed_1)})

#this will ignore the columns in removed tables
removed=final.loc[(final['TABLE_NAME'].isnull()) &( np.logical_not(final['col_name'].isin(ignore_list))) & ( np.logical_not(final['table'].isin(tab_removed_1))),['table','col_name','data_type']]



dtype_changes.to_excel(writer, sheet_name="dtype_changes", index=False)
removed.to_excel(writer, sheet_name="removed_fields", index=False)
#removed_ignoring_removed_tables.to_excel(writer, sheet_name="removed_fields_addn", index=False)
added.to_excel(writer, sheet_name="fields added", index=False)
tables_added.to_excel(writer, sheet_name="Tables added", index=False)
tab_removed.to_excel(writer, sheet_name="Tables removed", index=False)



#dummy=pd.DataFrame()
#dummy.to_excel(writer, sheet_name="Tables", index=False)
#worksheet = writer.sheets["Tables"]
#worksheet.write(2, 0, tab_added)
#worksheet.write(4, 0, tab_removed)

#writer.write_cells(tab_added,sheet_name="Tables", startrow=4, startcol=0)
#writer.write_cells(tab_removed,sheet_name="Tables", startrow=8, startcol=0)
writer.save()
writer.close()

In [28]:
stmnt='echo \"Schema changes for GCCT when we compare DSC current schema with DES '+output_name+'\" | mailx -a GCCT_schema_changes_'+output_name+'.xlsx -s GCCT\ Schema\ Changes\ DSC\ VS\ DES\ '+output_name+' latluri1@ford.com'

In [29]:
print(stmnt)

echo "Schema changes for GCCT when we compare DSC current schema with DES DESscript_20190313" | mailx -a GCCT_schema_changes_DESscript_20190313.xlsx -s GCCT\ Schema\ Changes\ DSC\ VS\ DES\ DESscript_20190313 latluri1@ford.com


In [30]:
commands.getstatusoutput(stmnt)

(0, '')